# Demonstration of the prediction models
Tensorflow models for redox potential and stability score of organic radicals

In [1]:
from collections import Counter

import numpy as np
import pandas as pd
import tensorflow as tf 
import nfp  # pip install nfp
import rdkit

The key libraries used by this notebook are listed above. Versions for `tensorflow` and `nfp` likely affect compatibility and are listed below

In [2]:
nfp.__version__  # github.com/NREL/nfp

'0.3.5'

In [3]:
tf.__version__

'2.6.0'

Load the preprocessor, which converts `rdkit.Chem.Mol` objects into sets of tensors understandable by the GNN prediction models

In [4]:
from models.preprocessor import load_preprocessor
preprocessor = load_preprocessor()

Load the tensorflow models for redox potential and stability

In [5]:
redox_model = tf.keras.models.load_model('models/redox_model', compile=False)
stability_model = tf.keras.models.load_model('models/stability_model', compile=False)

Here we define some convenience functions:

The **redox model** returns a length-2 array of reduction and oxidation potentials.

The **stability model** returns a list of two arrays:
* the first is a vector of predicted fractional spins on each heavy atom (sums to 1)
* the second is a vector of predicted buried volumes for each atom (in percent)

The ordering of the atom predictions is the same as the ordering of atoms in the input RDKit Mol object.

In [6]:
def check_radical(mol):
    """Ensures that the mol has a single formal radical center"""
    radical_count = Counter([atom.GetNumRadicalElectrons() for atom in mol.GetAtoms()])
    assert radical_count.keys() == {0, 1}    
    assert radical_count[1] == 1


def predict_redox(smiles):
    """Predicts (reduction, oxidation) potentials for a given SMILES string"""
    mol = rdkit.Chem.MolFromSmiles(smiles)
    check_radical(mol)
    return redox_model([np.expand_dims(val, 0) for val in 
                        preprocessor(mol).values()]).numpy().flatten()


def predict_stability(smiles):
    """Predicts the combined radical stability score for a given SMILES string"""
    
    mol = rdkit.Chem.MolFromSmiles(smiles)
    check_radical(mol)
    spins, buried_vol = stability_model([np.expand_dims(val, 0) for val in 
                                         preprocessor(mol).values()])

    max_spin_index = spins.numpy().argmax()
    max_spin = spins.numpy().flatten()[max_spin_index]
    buried_vol_at_max_spin = buried_vol.numpy().flatten()[max_spin_index]
    return 50 * (1 - max_spin) + buried_vol_at_max_spin

Here we take an example that has converged reduction and oxidation potentials

In [7]:
redox_df = pd.read_csv('data/radical_db_redox_potentials.csv.gz')
redox_df.iloc[2]

smiles             C#C/C(C)=C(/[CH2])C
reduction                     0.781466
oxidation                    -0.915292
stability_score              78.894325
Name: 2, dtype: object

In [8]:
reduction, oxidation = predict_redox('C#C/C(C)=C(/[CH2])C')
reduction, oxidation

(0.7709613, -0.92244977)

In [9]:
predict_stability('C#C/C(C)=C(/[CH2])C')

78.4802514910698